In [ ]:
def state_aggregate(stateobj, years = None, stacked = False):
    subset_dat = filter_years(stateobj, years)
    
    counts = subset_dat.groupby(stateobj.paths['departure'][0]).count()
    counts = counts.rename(stateobj.paths['departure'][1])
    counts = counts.iloc[:,0]
     
    order_of_outputs = ['Above Departure', 'Within Range', 'Below Range', 'Missing, Indeterminable, or Inapplicable']
    porportions_list = []
    for item in order_of_outputs:
        porportions_list.append(round((100 * counts.at[item]  /  subset_dat.shape[0]),1))
    
    if stacked:
        arr = np.array(porportions_list)
        arr = arr.T
        plot_departures_stacked([stateobj.name], porportions_list, [stateobj.name], order_of_outputs)
    else:
        plot_departures(order_of_outputs, porportions_list, [stateobj.name], s = False)
    
    return counts, porportions_list

In [ ]:
state_aggregate(states['minnesota'], stacked = False)

In [ ]:
def race_profile(stateobj, years = None, stacked = True):
    subset_dat = filter_years(stateobj, years)
    #grouping by departure and race
    #divide by count(race) for racial percentages, to get counts, remove the denominator
    counts = subset_dat.groupby([stateobj.paths['race'][0], stateobj.paths['departure'][0]]).count()
    perc = round(100 * counts / subset_dat.groupby([stateobj.paths['race'][0]]).count(), 1)
    
    # renames the values to have a description
    perc = perc.rename(stateobj.paths['race'][1], level = 0)
    perc = perc.rename(stateobj.paths['departure'][1], level = 1)
    perc = perc.iloc[:,0]  # all columns are the same, so we pull the first one
    
    counts = counts.rename(stateobj.paths['race'][1], level = 0)
    counts = counts.rename(stateobj.paths['departure'][1], level = 1)
    counts = counts.iloc[:,0]  # all columns are the same, so we pull the first one
    
    comb_df = pd.concat([counts,perc],axis=1)
    comb_df.columns = ['count', 'percent']
    
    order_of_outputs = ['Above Departure', 'Within Range', 'Below Range', 'Missing, Indeterminable, or Inapplicable']
    
    #get the order of the races four out output, may make this a function
    order_of_races = []
    for idx in perc.index:  #idx is a tuple of (race, departure_type)
        if idx[0] not in order_of_races:
            order_of_races.append(idx[0])
            
    # we want our data to be in the shape: number of levels in output (departure) x number of subgroups (race)        
    percentages = np.zeros((len(order_of_outputs), len(order_of_races)))
    for dep in range(len(order_of_outputs)):
        for race in range(len(order_of_races)):
            percentages[dep, race] = perc.loc[(order_of_races[race], order_of_outputs[dep]),]
    #warning, potential for keyerror above if sonehow we have a race that doesn't have all departure levels        
    print(order_of_races)
    print(type(order_of_races))
    if stacked:
        plot_departures_stacked(order_of_races, percentages, [stateobj.name, 'race'], order_of_outputs)
    else:
        for i in range(len(order_of_races)):
            plot_departures(order_of_outputs, percentages[:,i], [stateobj.name, order_of_races[i]])
    return comb_df

In [ ]:
race_profile(states['minnesota'], stacked = True)